In [1]:
from bs4 import BeautifulSoup as BeautifulSoup
import requests
import time
import pandas as pd
from pandas import DataFrame as df
from selenium import webdriver
from pyvirtualdisplay import Display

In [2]:
def scrape_list_webdriver(url):
    options = webdriver.ChromeOptions()
    options.add_experimental_option("excludeSwitches", ["ignore-certificate-errors"])
    driver = webdriver.Chrome(chrome_options=options)
    driver.get(url)
    time.sleep(3)
    soup_page = BeautifulSoup(driver.page_source, 'lxml')
    li = soup_page.find("div", {"class" : "col-xs-12 col-sm-9 col-sm-push-3 col-md-10 col-md-push-2 col-lg-10 col-lg-push-2"})
    therapists = []
    for item in li.find_all("div", {"class" : "col-xs-12 col-sm-12 col-md-12 col-lg-12"}):
        therapists.append(item)
#         if(len(therapists) == 7):
#             break
    url = []
    for item in therapists:
        therapist = item.find("div", {"class":"result-row normal-result"})
        if therapist != None:
            url.append(therapist['data-result-url'])
    driver.close()
    return (url, soup_page)

In [30]:
homepage = 'http://therapists.psychologytoday.com/rms'
# url1 = '/prof_results.php?sid=1492654955.5273_18235&state=MA&spec=335&rec_next=1'
# url2 = '/prof_results.php?sid=1492654955.5273_18235&state=MA&spec=335&rec_next=21'
# url3 = '/prof_results.php?sid=1492654955.5273_18235&state=MA&spec=335&rec_next=41'
# url4 = '/prof_results.php?sid=1492654955.5273_18235&state=MA&spec=335&rec_next=61'
# url5 = '/prof_results.php?sid=1492654955.5273_18235&state=MA&spec=335&rec_next=81'
# url6 = '/prof_results.php?sid=1492654955.5273_18235&state=MA&spec=335&rec_next=101'
# url_list1 = scrape_list_webdriver(url1)
# url_list2 = scrape_list_webdriver(url2)
# url_list3 = scrape_list_webdriver(url3)
# url_list4 = scrape_list_webdriver(url4)
# url_list5 = scrape_list_webdriver(url5)
# url_list6 = scrape_list_webdriver(url6)
def pages_scrape(soup_page, url):
#     options = webdriver.ChromeOptions()
#     options.add_experimental_option("excludeSwitches", ["ignore-certificate-errors"])
#     driver = webdriver.Chrome(chrome_options=options)
#     driver.get(url)
    time.sleep(3)
#     soup_page = BeautifulSoup(driver.page_source, 'lxml')
    pages_block = soup_page.find("div", {"class":"endresults-right text-right"})
    pages = ''
#     for item in pages_block.find_all('a'):
#         pages.append(item['href'])
    if pages_block != None:
        pages_list = pages_block.find_all('a')
        if len(pages_list) > 0 and pages_list[len(pages_list) - 1]['href'] != url:
            pages = pages_list[len(pages_list) - 1]['href']
    #driver.close()
    return pages

In [196]:
url_list = []
pages = homepage+'/prof_results.php?state=ID&spec=335'
# pages = pages_scrape(url)
# print(pages)
while pages != '':
    (url, soup_page) = scrape_list_webdriver(pages)
    url_list += url
    pages = pages_scrape(soup_page, pages)
# if(len(url_list) == 0):
#     url_list += scrape_list_webdriver(url)

In [197]:
# url_list = url_list1 + url_list2 + url_list3 + url_list4 + url_list5 + url_list6
len(url_list)
url_list = url_list[:5]
url_list

['https://therapists.psychologytoday.com/rms/prof_detail.php?profid=234896&ref=1&sid=1492787363.0517_24177&state=ID&spec=335',
 'https://therapists.psychologytoday.com/rms/prof_detail.php?profid=310107&ref=2&sid=1492787363.0517_24177&state=ID&spec=335',
 'https://therapists.psychologytoday.com/rms/prof_detail.php?profid=32706&ref=3&sid=1492787363.0517_24177&state=ID&spec=335',
 'https://therapists.psychologytoday.com/rms/prof_detail.php?profid=194809&ref=4&sid=1492787363.0517_24177&state=ID&spec=335',
 'https://therapists.psychologytoday.com/rms/prof_detail.php?profid=41799&ref=5&sid=1492787363.0517_24177&state=ID&spec=335']

In [198]:
#address1, address2, tel, specialities, Years, License # and state
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["ignore-certificate-errors"])
#for item in url:
#item = 'https://psychiatrists.psychologytoday.com/rms/prof_detail.php?profid=252042&ref=1&sid=1492538806.2691_9559&state=MA&spec=335'
#item = 'https://psychiatrists.psychologytoday.com/rms/prof_detail.php?profid=224544&ref=2&sid=1492538806.2691_9559&state=MA&spec=335'
def scrape_therapist(url):
    driver = webdriver.Chrome(chrome_options=options)
    driver.get(url)
    soup_page = BeautifulSoup(driver.page_source, 'lxml')
    time.sleep(3)
    address_block1 = soup_page.find("div", {"class":"address address-rank-1"})
    if address_block1 == None:
        return (None, None, None, None, None, None, None, None, None)
    addressInfo1 = address_block1.find("div", {"itemprop" : "address"})
    address1 = ''
    for item in addressInfo1.find_all('span'):
        address1 += item.text
        address1 += ' '
    address_block2 = soup_page.find("div", {"class" : "address address-rank-2"})
    
    address2 = ''
    if address_block2 != None:
        addressInfo2 = address_block2.find("div", {"itemprop" : "address"})
        address2 = ''
        for item in addressInfo2.find_all('span'):
            address2 += item.text
            address2 += ' '
        address2 = " ".join(address2.split())
    address1 = " ".join(address1.split())
    
    name = soup_page.find("h1", {"itemprop" : "name"}).text
    name = " ".join(name.split())
    
    tel = ''
    if addressInfo1.find('a') != None:
        tel += addressInfo1.find('a').text
    spec_block = soup_page.find("div", {"class" : "col-xs-12 col-sm-4 col-md-5 col-lg-5"}).find_all("div", {"class" : "spec-list"})
    
    specialties_block = None
    issues_block = None
    
    client_focus = ''
    for item in spec_block:
        if(item.find('h2') != None and item.find('h2').text == "Client Focus"):
            temp = item.find_all("div", {"class" : "row"})
            for tmp in temp:
                if tmp != None:
                    txt = " ".join(tmp.text.split())
                    if client_focus != '':
                        client_focus += ','
                    client_focus += txt
    
    if len(spec_block) > 0:
        specialties_block = spec_block[0]
    if len(spec_block) > 1:
        issues_block = spec_block[1]
    specialties = ''
    specialties_list = specialties_block.find("ul", {"class" : "specialties-list"})
    if specialties_list != None:
        for item in specialties_list.find_all('li'):
            if(len(specialties) > 0):
                specialties += ','
            specialties += item.text
    
    
    issurance_plans = ''
    issurance_block = soup_page.find_all("div", {"class" : "col-xs-12 col-sm-6 col-md-6 col-lg-6 col-tight-right"})
    for item in issurance_block:
        li = item.find_all('li')
        for i in li:
            txt = " ".join(i.text.split())
            if issurance_plans != '':
                issurance_plans += ','
            issurance_plans += txt
    
    Qualifications_block = soup_page.find("div", {"class" : "profile-qualifications"})

    Qualifications = Qualifications_block.find_all('li')
    Years = ''
    License_Num_And_State = ''
    for item in Qualifications:
        if(item.find('strong') != None and item.find('strong').text == 'Years in Practice:'):
            Years = item.text
            Years = " ".join(Years.split())
        if(item.find('strong') != None and item.find('strong').text == 'License No. and State:'):
            License_Num_And_State = item.text
            License_Num_And_State = " ".join(License_Num_And_State.split())
    driver.close()
    return (name, address1, address2, tel, specialties, Years, client_focus, License_Num_And_State, issurance_plans) # and state

In [199]:
columns = ['name', 'address', 'Additional address', 'telephone', 'specialties', 'years in practice', 'client_focus', 'License_Num_And_State', 'Accepted Issurance']
df = pd.DataFrame(columns=columns) 
for item in url_list:
    (name, address1, address2, tel, specialties, Years, client_focus, License_Num_And_State, issurance) = scrape_therapist(item)
    if address1 == None:
        continue
    df.loc[len(df.index)] = [name, address1, address2, tel, specialties, Years, client_focus, License_Num_And_State, issurance]
df

,name,address,Additional address,telephone,specialties,years in practice,client_focus,License_Num_And_State,Accepted Issurance
0,Ayako Otani Campion,410 S Orchard St Boise Idaho 83705,,(208) 475-4663,"Grief,Trauma and PTSD,Relationship Issues",,Preteens / Tweens (11 to 13) Adolescents / Tee...,License No. and State: LPC-5612 Idaho,"Accepts most insurance providers,Aetna,BlueCro..."
1,Crystal Maust,8590 W Fairview Ave Boise Idaho 83704,,(208) 423-1680,,,Adults Elders (65+),License No. and State: LPC-6280 Idaho,"Blue Cross,Medicaid"
2,Satu H. Woodland,3152 S Bown Way Suite #102 Boise Idaho 83706,,(208) 413-9203,"Depression,Anxiety,Psychosis",Years in Practice: 20+ Years,Children (6 to 10) Preteens / Tweens (11 to 13...,License No. and State: CNS-65-A Idaho,"Anthem,Blue Care Network,Blue Cross,Blue Shiel..."
3,Peter Billings,1412 W Washington St Boise Idaho 83702,,(208) 297-5474,"Relationship Issues,Marital and Premarital,Fam...",Years in Practice: 10 Years,Adolescents / Teenagers (14 to 19) Adults Elde...,License No. and State: LPC-3798 Idaho,
4,"Judy N Lam, PhD, PLLC","1602 W Hays Street, Suite 201, Free Parking Bo...",,(208) 994-5457,"Resilience, EMDR,Anxiety,Depression",Years in Practice: 20+ Years,Children (6 to 10) Adolescents / Teenagers (14...,License No. and State: PSY-202366 Idaho,"Aetna,BlueCross and BlueShield,Idaho Physician..."


In [200]:
df.to_csv('ID.csv')